In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/MyDrive/Colab Notebooks/tf_model'

/content/gdrive/MyDrive/Colab Notebooks/tf_model


In [3]:
import tensorflow as tf
import os
import pickle
import pandas as pd
import numpy as np
import multiprocessing
from itertools import repeat

In [5]:
with open("../X_noise.csv", "rb") as fp:   # Unpickling
   X = pickle.load(fp)

In [6]:
with open("../Y_noise.csv", "rb") as fp:   # Unpickling
   Y = pickle.load(fp)

In [7]:
with open("../Y_lbl_noise.csv", "rb") as fp:   # Unpickling
   Y_lbl = pickle.load(fp)

In [8]:
input_shape = X.shape
print(input_shape)

(20821, 1000, 12)


In [9]:
test_fold = 10
val_fold = 9
train_fold = 8
# Train
X_test = X[np.where(Y_lbl.strat_fold == test_fold)]
y_test= Y[(Y_lbl.strat_fold == test_fold)]
# Validation
X_val = X[np.where(Y_lbl.strat_fold == val_fold)]
y_val = Y[(Y_lbl.strat_fold == val_fold)]
# Test
X_train = X[np.where(Y_lbl.strat_fold <= train_fold)]
y_train = Y[Y_lbl.strat_fold <= train_fold]

In [10]:
print(X_train.shape, X_test.shape, X_val.shape)
print(y_train.shape, y_test.shape, y_val.shape)

(16593, 1000, 12) (2123, 1000, 12) (2105, 1000, 12)
(16593, 5) (2123, 5) (2105, 5)


# Bad Examples 

In [14]:
model = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(64, 5, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(5, activation='softmax')  
])

In [ ]:
model.compile(optimizer='adam', metrics='binary_accuracy'
              , loss=tf.keras.losses.binary_crossentropy)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=256,
    epochs=20,
)

Epoch 1/20
9/9 [==============================] - 13s 1s/step - loss: 0.6235 - binary_accuracy: 0.7353 - val_loss: 0.5552 - val_binary_accuracy: 0.7396
Epoch 2/20
9/9 [==============================] - 12s 1s/step - loss: 0.5420 - binary_accuracy: 0.7575 - val_loss: 0.5138 - val_binary_accuracy: 0.7687
Epoch 3/20
9/9 [==============================] - 12s 1s/step - loss: 0.4933 - binary_accuracy: 0.7741 - val_loss: 0.4594 - val_binary_accuracy: 0.7811
Epoch 4/20
9/9 [==============================] - 12s 1s/step - loss: 0.4456 - binary_accuracy: 0.7879 - val_loss: 0.4574 - val_binary_accuracy: 0.7787
Epoch 5/20
9/9 [==============================] - 12s 1s/step - loss: 0.4110 - binary_accuracy: 0.7984 - val_loss: 0.4301 - val_binary_accuracy: 0.8011
Epoch 6/20
9/9 [==============================] - 12s 1s/step - loss: 0.3822 - binary_accuracy: 0.8218 - val_loss: 0.4091 - val_binary_accuracy: 0.8099
Epoch 7/20
9/9 [==============================] - 12s 1s/step - loss: 0.3581 - binary_ac

In [15]:
model_small = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(32, 5, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(5, activation='softmax')  
])

In [16]:
model_small.compile(optimizer='adam', metrics='binary_accuracy'
              , loss=tf.keras.losses.binary_crossentropy)

In [17]:
history = model_small.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=256,
    epochs=15,
)

Epoch 1/15
67/67 [==============================] - 4s 49ms/step - loss: 0.4978 - binary_accuracy: 0.7704 - val_loss: 0.4071 - val_binary_accuracy: 0.8070
Epoch 2/15
67/67 [==============================] - 2s 23ms/step - loss: 0.3729 - binary_accuracy: 0.8249 - val_loss: 0.3633 - val_binary_accuracy: 0.8288
Epoch 3/15
67/67 [==============================] - 2s 23ms/step - loss: 0.3410 - binary_accuracy: 0.8402 - val_loss: 0.3508 - val_binary_accuracy: 0.8327
Epoch 4/15
67/67 [==============================] - 2s 23ms/step - loss: 0.3212 - binary_accuracy: 0.8496 - val_loss: 0.3435 - val_binary_accuracy: 0.8415
Epoch 5/15
67/67 [==============================] - 2s 23ms/step - loss: 0.3113 - binary_accuracy: 0.8544 - val_loss: 0.3264 - val_binary_accuracy: 0.8483
Epoch 6/15
67/67 [==============================] - 2s 23ms/step - loss: 0.2938 - binary_accuracy: 0.8616 - val_loss: 0.3206 - val_binary_accuracy: 0.8485
Epoch 7/15
67/67 [==============================] - 2s 23ms/step - los

In [21]:
model2 = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(8, 3, activation='tanh', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(8, 3, activation='tanh', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(16, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(16, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=4, strides=2),
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(5, activation='softmax')  
])

In [22]:
model2.compile(optimizer='adam', 
               metrics='binary_accuracy',
               loss=tf.keras.losses.binary_crossentropy)

In [23]:
history = model2.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    #batch_size=256,
    epochs=8,
)

Epoch 1/8
535/535 [==============================] - 5s 7ms/step - loss: 0.4232 - binary_accuracy: 0.8003 - val_loss: 0.3677 - val_binary_accuracy: 0.8271
Epoch 2/8
535/535 [==============================] - 3s 6ms/step - loss: 0.3424 - binary_accuracy: 0.8429 - val_loss: 0.3519 - val_binary_accuracy: 0.8323
Epoch 3/8
535/535 [==============================] - 3s 6ms/step - loss: 0.3120 - binary_accuracy: 0.8545 - val_loss: 0.3286 - val_binary_accuracy: 0.8460
Epoch 4/8
535/535 [==============================] - 3s 6ms/step - loss: 0.2910 - binary_accuracy: 0.8627 - val_loss: 0.3242 - val_binary_accuracy: 0.8506
Epoch 5/8
535/535 [==============================] - 3s 6ms/step - loss: 0.2690 - binary_accuracy: 0.8698 - val_loss: 0.3341 - val_binary_accuracy: 0.8468
Epoch 6/8
535/535 [==============================] - 3s 6ms/step - loss: 0.2494 - binary_accuracy: 0.8771 - val_loss: 0.3377 - val_binary_accuracy: 0.8465
Epoch 7/8
535/535 [==============================] - 3s 6ms/step - los

# Parameters tuning

In [ ]:
# hyper parameters
activation1 = tf.keras.activations.sigmoid
activation2 = tf.keras.activations.tanh
activation3 = tf.keras.activations.relu

kernel_sizes = [3, 5, 7]
poolings = [max, avg]
pooling_sizes = [2, 3, 4]

In [20]:
model_kernel_size3 = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(128, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(128, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(5, activation='softmax')  
])

In [21]:
model_kernel_size3.compile(optimizer='adam', 
               metrics='binary_accuracy',
               loss=tf.keras.losses.binary_crossentropy)

In [22]:
history_ks3 = model_kernel_size3.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    #batch_size=256,
    epochs=8,
)

Epoch 1/8
519/519 [==============================] - 6s 10ms/step - loss: 0.3899 - binary_accuracy: 0.8190 - val_loss: 0.3398 - val_binary_accuracy: 0.8424
Epoch 2/8
519/519 [==============================] - 5s 9ms/step - loss: 0.3062 - binary_accuracy: 0.8578 - val_loss: 0.3196 - val_binary_accuracy: 0.8540
Epoch 3/8
519/519 [==============================] - 5s 9ms/step - loss: 0.2743 - binary_accuracy: 0.8684 - val_loss: 0.3123 - val_binary_accuracy: 0.8539
Epoch 4/8
519/519 [==============================] - 5s 9ms/step - loss: 0.2510 - binary_accuracy: 0.8766 - val_loss: 0.3263 - val_binary_accuracy: 0.8538
Epoch 5/8
519/519 [==============================] - 5s 9ms/step - loss: 0.2205 - binary_accuracy: 0.8855 - val_loss: 0.3454 - val_binary_accuracy: 0.8543
Epoch 6/8
519/519 [==============================] - 5s 9ms/step - loss: 0.1789 - binary_accuracy: 0.8989 - val_loss: 0.3859 - val_binary_accuracy: 0.8529
Epoch 7/8
519/519 [==============================] - 5s 9ms/step - lo

In [11]:
model_kernel_size5 = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(32, 5, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(32, 5, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(64, 5, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(64, 5, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(128, 5, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(128, 5, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(5, activation='softmax')  
])

In [12]:
model_kernel_size5.compile(optimizer='adam', 
               metrics='binary_accuracy',
               loss=tf.keras.losses.binary_crossentropy)

In [13]:
history_ks5 = model_kernel_size5.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    #batch_size=256,
    epochs=8,
)

Epoch 1/8
519/519 [==============================] - 38s 11ms/step - loss: 0.3811 - binary_accuracy: 0.8250 - val_loss: 0.3368 - val_binary_accuracy: 0.8375
Epoch 2/8
519/519 [==============================] - 5s 10ms/step - loss: 0.3012 - binary_accuracy: 0.8602 - val_loss: 0.3103 - val_binary_accuracy: 0.8562
Epoch 3/8
519/519 [==============================] - 5s 10ms/step - loss: 0.2728 - binary_accuracy: 0.8693 - val_loss: 0.3084 - val_binary_accuracy: 0.8592
Epoch 4/8
519/519 [==============================] - 5s 10ms/step - loss: 0.2526 - binary_accuracy: 0.8761 - val_loss: 0.2911 - val_binary_accuracy: 0.8619
Epoch 5/8
519/519 [==============================] - 5s 10ms/step - loss: 0.2332 - binary_accuracy: 0.8814 - val_loss: 0.3134 - val_binary_accuracy: 0.8581
Epoch 6/8
519/519 [==============================] - 5s 10ms/step - loss: 0.2094 - binary_accuracy: 0.8890 - val_loss: 0.3089 - val_binary_accuracy: 0.8594
Epoch 7/8
519/519 [==============================] - 5s 10ms/st

In [32]:
import time
a = time.time()
y_pred = model_kernel_size5.predict(X_test)
b = time.time()
timing = (b - a)
bin_ce, bin_acc = 0, 0
for i in range(y_pred.shape[0]):
  bin_ce += tf.keras.metrics.binary_crossentropy(y_test[i], y_pred[i])
  bin_acc += tf.keras.metrics.binary_accuracy(y_test[i], y_pred[i])
bin_ce /= y_pred.shape[0]
bin_acc /= y_pred.shape[0]
print(f'Total time:{timing}\nTest data length: {X_test.shape[0]}\nbinary_cross_entropy_loss: {bin_ce}\nbinary_accuracy: {bin_acc}')


Total time:0.2991194725036621
Test data length: 2123
binary_cross_entropy_loss: 0.647589921951294
binary_accuracy: 0.8487984538078308


In [18]:
model_kernel_size7 = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(32, 7, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(32, 7, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(64, 7, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(64, 7, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(128, 7, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(128, 7, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(5, activation='softmax')  
])

In [19]:
model_kernel_size7.compile(optimizer='adam', 
               metrics='binary_accuracy',
               loss=tf.keras.losses.binary_crossentropy)

In [20]:
history_ks7 = model_kernel_size7.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    #batch_size=256,
    epochs=8,
)

Epoch 1/8
519/519 [==============================] - 7s 11ms/step - loss: 0.3776 - binary_accuracy: 0.8241 - val_loss: 0.3252 - val_binary_accuracy: 0.8458
Epoch 2/8
519/519 [==============================] - 5s 11ms/step - loss: 0.3001 - binary_accuracy: 0.8607 - val_loss: 0.3108 - val_binary_accuracy: 0.8545
Epoch 3/8
519/519 [==============================] - 6s 11ms/step - loss: 0.2747 - binary_accuracy: 0.8696 - val_loss: 0.3020 - val_binary_accuracy: 0.8572
Epoch 4/8
519/519 [==============================] - 6s 11ms/step - loss: 0.2582 - binary_accuracy: 0.8747 - val_loss: 0.2942 - val_binary_accuracy: 0.8600
Epoch 5/8
519/519 [==============================] - 6s 11ms/step - loss: 0.2373 - binary_accuracy: 0.8817 - val_loss: 0.3008 - val_binary_accuracy: 0.8611
Epoch 6/8
519/519 [==============================] - 6s 11ms/step - loss: 0.2127 - binary_accuracy: 0.8876 - val_loss: 0.3077 - val_binary_accuracy: 0.8588
Epoch 7/8
519/519 [==============================] - 6s 11ms/ste

In [21]:
model_act_sigmoid = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(32, 3, activation='sigmoid', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(32, 3, activation='sigmoid', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(64, 3, activation='sigmoid', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(64, 3, activation='sigmoid', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(128, 3, activation='sigmoid', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(128, 3, activation='sigmoid', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='sigmoid'),
      tf.keras.layers.Dense(5, activation='softmax')  
])

In [22]:
model_act_sigmoid.compile(optimizer='adam', 
               metrics='binary_accuracy',
               loss=tf.keras.losses.binary_crossentropy)

In [23]:
history_act_sigmoid = model_act_sigmoid.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    #batch_size=256,
    epochs=8,
)

Epoch 1/8
519/519 [==============================] - 6s 10ms/step - loss: 0.5560 - binary_accuracy: 0.7388 - val_loss: 0.5459 - val_binary_accuracy: 0.7401
Epoch 2/8
519/519 [==============================] - 5s 9ms/step - loss: 0.5454 - binary_accuracy: 0.7400 - val_loss: 0.5459 - val_binary_accuracy: 0.7401
Epoch 3/8
519/519 [==============================] - 5s 9ms/step - loss: 0.5458 - binary_accuracy: 0.7393 - val_loss: 0.5455 - val_binary_accuracy: 0.7401
Epoch 4/8
519/519 [==============================] - 5s 9ms/step - loss: 0.5459 - binary_accuracy: 0.7399 - val_loss: 0.5460 - val_binary_accuracy: 0.7401
Epoch 5/8
519/519 [==============================] - 5s 9ms/step - loss: 0.5455 - binary_accuracy: 0.7401 - val_loss: 0.5452 - val_binary_accuracy: 0.7401
Epoch 6/8
519/519 [==============================] - 5s 9ms/step - loss: 0.5456 - binary_accuracy: 0.7395 - val_loss: 0.5478 - val_binary_accuracy: 0.7401
Epoch 7/8
519/519 [==============================] - 5s 9ms/step - lo

In [32]:
model_act_tanh = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(32, 5, activation='tanh', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(32, 5, activation='tanh', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(64, 5, activation='tanh', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(64, 5, activation='tanh', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(128, 5, activation='tanh', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(128, 5, activation='tanh', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='tanh'),
      tf.keras.layers.Dense(5, activation='softmax')  
])

In [33]:
model_act_tanh.compile(optimizer='adam', 
               metrics='binary_accuracy',
               loss=tf.keras.losses.binary_crossentropy)

In [34]:
history_act_tanh = model_act_tanh.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    #batch_size=256,
    epochs=15,
)

Epoch 1/15
519/519 [==============================] - 6s 11ms/step - loss: 0.4184 - binary_accuracy: 0.8095 - val_loss: 0.3768 - val_binary_accuracy: 0.8291
Epoch 2/15
519/519 [==============================] - 5s 10ms/step - loss: 0.3465 - binary_accuracy: 0.8399 - val_loss: 0.3662 - val_binary_accuracy: 0.8238
Epoch 3/15
519/519 [==============================] - 5s 10ms/step - loss: 0.3270 - binary_accuracy: 0.8486 - val_loss: 0.3629 - val_binary_accuracy: 0.8340
Epoch 4/15
519/519 [==============================] - 5s 10ms/step - loss: 0.3221 - binary_accuracy: 0.8526 - val_loss: 0.3334 - val_binary_accuracy: 0.8458
Epoch 5/15
519/519 [==============================] - 5s 10ms/step - loss: 0.3130 - binary_accuracy: 0.8548 - val_loss: 0.3517 - val_binary_accuracy: 0.8448
Epoch 6/15
519/519 [==============================] - 5s 10ms/step - loss: 0.3118 - binary_accuracy: 0.8559 - val_loss: 0.3498 - val_binary_accuracy: 0.8365
Epoch 7/15
519/519 [==============================] - 5s 1

In [35]:
model_avg_pool = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.AveragePooling1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.AveragePooling1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(128, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(128, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.AveragePooling1D(pool_size=2, strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(5, activation='softmax')  
])

In [36]:
model_avg_pool.compile(optimizer='adam', 
               metrics='binary_accuracy',
               loss=tf.keras.losses.binary_crossentropy)

In [37]:
history_avg_pool = model_avg_pool.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    #batch_size=256,
    epochs=8,
)

Epoch 1/8
519/519 [==============================] - 6s 10ms/step - loss: 0.3969 - binary_accuracy: 0.8160 - val_loss: 0.3368 - val_binary_accuracy: 0.8417
Epoch 2/8
519/519 [==============================] - 5s 9ms/step - loss: 0.3109 - binary_accuracy: 0.8560 - val_loss: 0.3114 - val_binary_accuracy: 0.8565
Epoch 3/8
519/519 [==============================] - 5s 9ms/step - loss: 0.2799 - binary_accuracy: 0.8673 - val_loss: 0.3166 - val_binary_accuracy: 0.8552
Epoch 4/8
519/519 [==============================] - 5s 9ms/step - loss: 0.2608 - binary_accuracy: 0.8732 - val_loss: 0.3081 - val_binary_accuracy: 0.8567
Epoch 5/8
519/519 [==============================] - 5s 9ms/step - loss: 0.2378 - binary_accuracy: 0.8814 - val_loss: 0.3278 - val_binary_accuracy: 0.8566
Epoch 6/8
519/519 [==============================] - 5s 9ms/step - loss: 0.2152 - binary_accuracy: 0.8866 - val_loss: 0.3308 - val_binary_accuracy: 0.8565
Epoch 7/8
519/519 [==============================] - 5s 9ms/step - lo

In [38]:
model_avg_pool5 = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.AveragePooling1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.AveragePooling1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(128, 3, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(128, 3, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.AveragePooling1D(pool_size=2, strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(5, activation='softmax')  
])

In [39]:
model_avg_pool5.compile(optimizer='adam', 
               metrics='binary_accuracy',
               loss=tf.keras.losses.binary_crossentropy)

In [40]:
history_avg_pool5 = model_avg_pool5.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    #batch_size=256,
    epochs=8,
)

Epoch 1/8
519/519 [==============================] - 6s 10ms/step - loss: 0.4048 - binary_accuracy: 0.8127 - val_loss: 0.3466 - val_binary_accuracy: 0.8393
Epoch 2/8
519/519 [==============================] - 5s 9ms/step - loss: 0.3155 - binary_accuracy: 0.8538 - val_loss: 0.3226 - val_binary_accuracy: 0.8521
Epoch 3/8
519/519 [==============================] - 5s 9ms/step - loss: 0.2850 - binary_accuracy: 0.8656 - val_loss: 0.3139 - val_binary_accuracy: 0.8546
Epoch 4/8
519/519 [==============================] - 5s 10ms/step - loss: 0.2659 - binary_accuracy: 0.8723 - val_loss: 0.3213 - val_binary_accuracy: 0.8539
Epoch 5/8
519/519 [==============================] - 6s 11ms/step - loss: 0.2436 - binary_accuracy: 0.8788 - val_loss: 0.3189 - val_binary_accuracy: 0.8537
Epoch 6/8
519/519 [==============================] - 6s 11ms/step - loss: 0.2202 - binary_accuracy: 0.8848 - val_loss: 0.3408 - val_binary_accuracy: 0.8523
Epoch 7/8
519/519 [==============================] - 6s 11ms/step 

In [41]:
model_pool_size4 = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(32, 5, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(32, 5, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(64, 5, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(64, 5, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Conv1D(128, 5, activation='relu', input_shape=input_shape[1:]), 
      tf.keras.layers.Conv1D(128, 5, activation='relu', input_shape=input_shape[1:]),
      tf.keras.layers.MaxPool1D(pool_size=2, strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(5, activation='softmax')  
])

In [43]:
model_pool_size4.compile(optimizer='adam', 
               metrics='binary_accuracy',
               loss=tf.keras.losses.binary_crossentropy)

In [44]:
history_avg_pool4 = model_avg_pool5.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    #batch_size=256,
    epochs=8,
)

Epoch 1/8
519/519 [==============================] - 5s 10ms/step - loss: 0.1284 - binary_accuracy: 0.9128 - val_loss: 0.5135 - val_binary_accuracy: 0.8486
Epoch 2/8
519/519 [==============================] - 5s 10ms/step - loss: 0.0908 - binary_accuracy: 0.9226 - val_loss: 0.5828 - val_binary_accuracy: 0.8438
Epoch 3/8
519/519 [==============================] - 5s 10ms/step - loss: 0.0590 - binary_accuracy: 0.9311 - val_loss: 0.7583 - val_binary_accuracy: 0.8429
Epoch 4/8
519/519 [==============================] - 5s 10ms/step - loss: 0.0471 - binary_accuracy: 0.9345 - val_loss: 0.8033 - val_binary_accuracy: 0.8431
Epoch 5/8
519/519 [==============================] - 5s 10ms/step - loss: 0.0364 - binary_accuracy: 0.9357 - val_loss: 0.8854 - val_binary_accuracy: 0.8418
Epoch 6/8
519/519 [==============================] - 5s 10ms/step - loss: 0.0306 - binary_accuracy: 0.9376 - val_loss: 0.9648 - val_binary_accuracy: 0.8412
Epoch 7/8
519/519 [==============================] - 5s 10ms/ste